In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This will likely need to change everytime if your path looks different to mine... thats annoying

In [59]:
PATH_TO_PROJECT = 'drive/Othercomputers/My MacBook Pro/FAERS-analytics'

In [60]:
%cd $PATH_TO_PROJECT

[Errno 2] No such file or directory: 'drive/Othercomputers/My MacBook Pro/FAERS-analytics'
/content/drive/Othercomputers/My MacBook Pro/FAERS-analytics


In [61]:
!ls

 data   data-out  'FAERS analytics.ipynb'   README.md


In [62]:
import os

output_path_dir = 'data-out/'
input_path_dir = 'data/'

if not os.path.isdir(output_path_dir):
  !mkdir $output_path_dir

for file in os.listdir(input_path_dir):
  if file.endswith(".zip"):
    input_path_file = input_path_dir + file
    output_path_file = output_path_dir + file.split('.zip')[0]
    if not os.path.isdir(output_path_file):
      !mkdir $output_path_file
      !unzip $input_path_file -d $output_path_file

In [63]:
!ls $output_path_dir

FAERS_ASCII_2023_Q3  FAERS_ASCII_2023_Q4
